In [52]:
"""
Decision tree clasifier
"""
import math
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


class Node:
    """
    Node is the clas that represents one leave of the tree
    """

    def __init__(self, gini, feature_index, threshold, left, right, combined_data):
        self.gini = gini
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.combined_data = combined_data

    def __str__(self):
        return f"{self.threshold}"


# Implement a decision tree classifier
class MyDecisionTreeClassifier:
    """
    Class of the tree that
    1) Has gini index function
    2) Has split data function
    """

    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.treeee = object

    def gini(self, groups):
        """
        A Gini score gives an idea of how good a split is by how mixed the
        classes are in the two groups created by the split.

        A perfect separation results in a Gini score of 0,
        whereas the worst case split that results in 50/50
        classes in each group result in a Gini score of 0.5
        (for a 2 class problem).
        """
        all_types = {}
        gini = 1
        for elem in groups:
            if elem[1] not in all_types:
                all_types[elem[1]] = 1
            else:
                all_types[elem[1]] += 1
        for item in all_types.values():
            gini -= (int(item) / len(groups)) ** 2
        return gini

    def split_data(self, combined_data) -> tuple[int, int]:
        """
        Function that finds the best question to
        split the tree and get as much unformation as possible.
        """
        # test all the possible splits in O(N*F) where N in number of samples
        # and F is number of features

        # return index and threshold value

        max_gain_index = 0
        basic_impurity = self.gini(combined_data)
        true_impurity = 0
        false_impurity = 0
        avg_impurity = 0
        feature_index = 0
        feature_value = 0
        for i, _ in enumerate(combined_data):
            for j, _ in enumerate(combined_data[0][0]):
                true_list = []
                false_list = []
                for elem in combined_data:
                    if elem[0][j] > combined_data[i][0][j]:
                        true_list.append(elem)
                    else:
                        false_list.append(elem)

                true_impurity = self.gini(true_list)
                false_impurity = self.gini(false_list)
                avg_impurity = (
                    len(true_list) / len(combined_data)
                ) * true_impurity + false_impurity * (
                    len(false_list) / len(combined_data)
                )
                if max_gain_index < basic_impurity - avg_impurity:
                    max_gain_index = basic_impurity - avg_impurity
                    feature_index = j
                    feature_value = combined_data[i][0][j]
        return (max_gain_index, feature_index, feature_value)

    def build_tree(self, combined_data, depth=0):
        """
        Builds Tree
        """
        max_gain_index, feature_index, feature_value = self.split_data(combined_data)
        if depth < self.max_depth and max_gain_index != 0:
            true_list = []
            false_list = []
            for elem in combined_data:
                if elem[0][feature_index] > feature_value:
                    true_list.append(elem)
                else:
                    false_list.append(elem)
            left_tree = self.build_tree(true_list, depth=depth + 1)
            right_tree = self.build_tree(false_list, depth=depth + 1)
            return Node(
                max_gain_index,
                feature_index,
                feature_value,
                left_tree,
                right_tree,
                combined_data,
            )
        return Node(max_gain_index, 0, 0, 0, 0, combined_data)

    def fit(self, x_list, y_list):
        """
        Function that train tree with data
        """
        self.treeee = self.build_tree(list(zip(x_list, y_list)))

    def predict(self, x_test):
        """
        Evaluate accuracy
        """
        predictions_list = []
        for test in x_test:
            current_node = self.treeee
            leaves_data = []
            possibility_dict = {}
            while True:
                if test[current_node.feature_index] > current_node.threshold:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
                if not current_node.right and not current_node.left:
                    break
            leaves_data = current_node.combined_data
            for data in leaves_data:
                if data[1] in possibility_dict:
                    possibility_dict[data[1]] += 1
                else:
                    possibility_dict[data[1]] = 1
            predictions_list.append(max(possibility_dict))
        return predictions_list

    def evaluate(self, x_test, y_test):
        """
        Evaluate accuracy
        """
        return sum(self.predict(x_test) == y_test) / len(y_test)


iris = load_iris()
x = iris["data"]
y = iris["target"]
x, x_test, y, y_test = train_test_split(x, y, test_size=0.3)

treeee = MyDecisionTreeClassifier(10)
treeee.fit(x_test, y_test)
print(treeee.evaluate(x,y))


0.9523809523809523
